In [64]:
import numpy as np
import tensorflow as tf
from sklearn import model_selection
import pandas as pd
import glob

In [52]:
def features_from_bottles(bottles):
    X = []
    y = []
    for bottle in bottles:
        label = bottle.split("/")[-1].replace("_inception_v3.txt", "")
        line = open(bottle).readlines()
        X.append( np.loadtxt(line[0].split(','), dtype=np.float32) )
        y.append(label)
        
    X = np.array(X)
    return (X, y)
        

In [57]:
bottles = glob.glob('./features/train_img_vectors/*.txt')
X, y = features_from_bottles(bottles)

In [58]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4)

In [87]:
train_labels = pd.read_csv('./DL3 Dataset/meta-data/train.csv')
test_labels = pd.read_csv('./DL3 Dataset/meta-data/test.csv')
train_labels = train_labels.set_index('Image_name');

In [88]:
train_labels.head(3)

,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,attrib_10,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
Image_name,,,,,,,,,,,,,,,,,,,,,
Image-1.jpg,0,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
Image-2.jpg,1,0,0,1,0,0,1,1,0,0,...,1,1,1,1,0,0,0,1,0,1
Image-3.jpg,1,0,0,1,0,0,0,1,1,0,...,1,0,0,1,0,1,0,1,0,1


In [94]:
len(train_labels.loc['Image-22.jpg'].tolist())

85

In [101]:
y_train_ = np.array([train_labels.loc[label] for label in y_train])
y_test_ = np.array([train_labels.loc[label] for label in y_test])